In [4]:
from py2neo import Graph, Node, Relationship, NodeMatcher
import json

NEO4J_URI = "neo4j+s://0753e756.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "6TfJUlBpkBs4OXZQUug_7kVqBXJi23vOHU39z7NoMf4"

graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [7]:
with open("LLMforgraph.json", "r") as f:
    data = json.load(f)

data

[{'section_title': 'Overview - Prepaid Phones',
  'tags': ['phones', 'prepaid', 'Straight Talk'],
  'content_type': 'landing_page',
  'source_url': 'https://www.straighttalk.com/all-phones',
  'availability': '',
  'review_summary': '',
  'content': 'This page appears to list various prepaid phone options provided by Straight Talk Wireless. It likely includes both basic phones and smartphones. The product offerings may vary by availability and current promotions.'},
 {'section_title': 'Promotional Offer - Free Motorola Edge',
  'tags': ['promotion', 'motorola', 'phones'],
  'content_type': 'promotion',
  'source_url': 'https://www.straighttalk.com/all-phones',
  'availability': '',
  'review_summary': '',
  'content': 'New and existing customers might be eligible to receive a free Motorola Edge 2024 phone if they purchase a 3-month Extended Silver Plan or a higher-tier plan. Itâ€™s important to check the specific terms and conditions, as availability and eligibility requirements may ap

In [8]:
matcher = NodeMatcher(graph)

for idx, entry in enumerate(data, start=1):
    section_id = idx
    section_title = entry["section_title"]
    content = entry["content"]
    url = entry["source_url"]
    content_type = entry["content_type"]
    tags = entry["tags"]

    section_node = Node("Section", id=section_id, title=section_title, content=content, url=url)
    graph.merge(section_node, "Section", "id")

    source_node = matcher.match("Source", url=url).first()
    if not source_node:
        source_node = Node("Source", url=url)
        graph.create(source_node)

    type_node = matcher.match("ContentType", type=content_type).first()
    if not type_node:
        type_node = Node("ContentType", type=content_type)
        graph.create(type_node)

    graph.merge(Relationship(section_node, "USES_SOURCE", source_node))
    graph.merge(Relationship(section_node, "HAS_TYPE", type_node))

    for tag in tags:
        tag_node = matcher.match("Tag", name=tag).first()
        if not tag_node:
            tag_node = Node("Tag", name=tag)
            graph.create(tag_node)

        graph.merge(Relationship(section_node, "HAS_TAG", tag_node))

print("Ingestion completed successfully.")

Ingestion completed successfully.


In [5]:
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship, NodeMatcher

NEO4J_URI = "neo4j+s://844a46bb.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pZutxY9g53VADs3yv2ycC5s-ftUbeWW9aqHjltYzPzc"

In [6]:
from py2neo import Graph, Node, Relationship

graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

def load_triples(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if not line:
            continue
        try:
            triple = eval(line)
            if len(triple) == 3:
                subject, predicate, obj = triple

                subj_node = Node("Entity", name=subject)
                obj_node = Node("Entity", name=obj)

                graph.merge(subj_node, "Entity", "name")
                graph.merge(obj_node, "Entity", "name")

                rel = Relationship(subj_node, predicate.upper().replace(" ", "_"), obj_node)
                graph.merge(rel)
        except Exception as e:
            print(f"Error processing line: {line}\nError: {e}")

load_triples("SPOTriples_openai.txt")
